In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf  
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten,
                                     Dense, Dropout, BatchNormalization)
from tensorflow.keras.mixed_precision import set_global_policy
import os
from PIL import Image


In [2]:
tf.config.set_visible_devices([], 'GPU')  # This hides all GPUs from TensorFlow.

# Now when you use TensorFlow, it will only use the CPU.

In [3]:
# tf.keras.backend.clear_session()
# set_global_policy('mixed_float16')
# gpus = tf.config.experimental.list_physical_devices('GPU')
# gpus

In [4]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         # Restrict TensorFlow to only allocate a specific amount of memory on the first GPU
#         tf.config.experimental.set_virtual_device_configuration(
#             gpus[0],
#             [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) # Adjust the memory limit as appropriate
#     except RuntimeError as e:
#         print(e)

In [5]:
import os
from PIL import Image
import numpy as np

# Define the path to the directory containing the folders
base_path = '../Preprocessed Dataset'

# List the folders in the directory
folders = ['IBM Plex Sans Arabic', 'Lemonada', 'Marhey', 'Scheherazade New']

# Create a dictionary to hold the labels for each folder
labels = {folder: i for i, folder in enumerate(folders)}

# Prepare a list to store the image data and labels
data = []

# Loop through each folder and each image within the folder
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpeg'):  # assuming the images are in PNG format
            image_path = os.path.join(folder_path, filename)
            # Load the image
            image = Image.open(image_path)
            # Optionally, convert the image to 'L' to ensure it's in grayscale
            if image.mode != 'L':
                image = image.convert('L')
            # Convert image data to array
            image_data = np.array(image)
            # Binarize the image data (0 or 1) directly instead of normalization to [0, 1] range
            image_data = (image_data > 127).astype(np.int16)  # Assuming binary threshold at the middle (127)
            # Append the image data and label to the list
            data.append((image_data, labels[folder]))

# Now, 'data' contains tuples of (image_data, label) where image_data are integers (0 or 1)


In [6]:
# Assuming 'data' is the list that contains all your images and their labels
# Separate data into two lists: features and labels
data, labels = zip(*data)

# Convert these lists to numpy arrays for compatibility with sklearn
data = np.array(data)
labels = np.array(labels)

# Split the data using sklearn's train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
data = 1
labels = 1

# Check the size of the training and test sets
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

Training set size: 3200
Test set size: 800


In [7]:
X_train[0].shape

(1181, 1181)

In [8]:
model = Sequential([
    # First convolution layer with input shape specified as (1181, 1181, 1)
    Conv2D(32, (3, 3), activation='relu', input_shape=(1181, 1181, 1)),
    MaxPooling2D((2, 2)),
    
    # Second convolution layer
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Third convolution layer
    Conv2D(48, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Fourth convolution layer
    Conv2D(48, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Flattening the output of the last pooling layer to feed it into the dense layer
    Flatten(),
    
    # Dense layers with dropout for regularization
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    
    # Output layer with softmax activation for multi-class classification
    Dense(4, activation='softmax')
])


In [9]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [10]:
# Train the model
history = model.fit(X_train[:1000], y_train[:1000], epochs=5)

Epoch 1/5


: 